In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os

In [2]:
# Konfigurasi dataset path
base_dir = "./dataset_padang_food"
img_height, img_width = 224, 224  # Ukuran gambar
batch_size = 32  # Ukuran batch

In [3]:
# Data Augmentation dan Pembagian Data
datagen = ImageDataGenerator(
    rescale=1.0/255,              # Normalisasi gambar
    validation_split=0.2,         # 80% training, 20% validation
    rotation_range=20,            # Rotasi gambar
    horizontal_flip=True          # Flip gambar horizontal
)

In [6]:
train_data = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

Found 799 images belonging to 9 classes.


In [7]:
val_data = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 194 images belonging to 9 classes.


In [8]:
# Load model pre-trained (MobileNetV2)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

9406464/9406464 [==============================] - 0s 0us/step


In [9]:
# Freeze layer pre-trained
base_model.trainable = False

In [10]:
# Tambahkan layer custom (head) untuk klasifikasi
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')  # Output sesuai jumlah kelas
])

In [11]:
# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [12]:
# Training model
epochs = 10
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)

Epoch 1/10
25/25 [==============================] - 24s 612ms/step - loss: 1.5919 - accuracy: 0.4543 - val_loss: 0.6931 - val_accuracy: 0.8557
Epoch 2/10
25/25 [==============================] - 9s 364ms/step - loss: 0.6886 - accuracy: 0.7785 - val_loss: 0.4699 - val_accuracy: 0.8660
Epoch 3/10
25/25 [==============================] - 9s 346ms/step - loss: 0.4657 - accuracy: 0.8461 - val_loss: 0.4681 - val_accuracy: 0.8454
Epoch 4/10
25/25 [==============================] - 9s 363ms/step - loss: 0.3640 - accuracy: 0.8861 - val_loss: 0.4132 - val_accuracy: 0.8299
Epoch 5/10
25/25 [==============================] - 9s 360ms/step - loss: 0.2623 - accuracy: 0.9186 - val_loss: 0.3663 - val_accuracy: 0.8866
Epoch 6/10
25/25 [==============================] - 9s 355ms/step - loss: 0.2249 - accuracy: 0.9324 - val_loss: 0.3745 - val_accuracy: 0.8918
Epoch 7/10
25/25 [==============================] - 9s 343ms/step - loss: 0.1799 - accuracy: 0.9462 - val_loss: 0.3261 - val_accuracy: 0.9072
Epoch

In [14]:
# Simpan model ke format TFLite
model.save("padang_food_model.h5")

In [15]:
# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Jef\AppData\Local\Temp\tmpnk3mjxvt\assets


INFO:tensorflow:Assets written to: C:\Users\Jef\AppData\Local\Temp\tmpnk3mjxvt\assets


In [16]:
# Simpan model TFLite
with open("padang_food_model.tflite", "wb") as f:
    f.write(tflite_model)

In [17]:
# Uji prediksi pada data baru (opsional)
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

In [18]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = train_data.class_indices
    label_map = {v: k for k, v in predicted_class.items()}
    return label_map[np.argmax(predictions)]